In [19]:
import sys
import gzip
import time
import random
import argparse
import numpy as np

import torch
from torch import nn
from torch import autograd
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as f

In [22]:
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from misc import *
from model import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
args = {'average':1, 'batch_size':16, 'embeddings': '../../data/askubuntu/vector/vectors_pruned.200.txt.gz',\
       'corpus': '../../data/askubuntu/text_tokenized.txt.gz', 'train':'../../data/askubuntu/train_random.txt',\
       'test': '../../data/askubuntu/test.txt', 'dev': '../../data/askubuntu/dev.txt', 'hidden_dim':100,\
       'embed_dim': 200, 'learning_rate': 0.001, 'l2_reg':1e-5, 'depth':1, 'dropout': 0.0, 'max_epoch':20,\
       'max_seq_len': 100, 'margin':0.3, 'reweight':0, 'layer':'lstm', 'kernel_size':3, 'save_model':0}

In [41]:
argparser = argparse.ArgumentParser()
#parser.parse_args(['--help'])
argparser.add_argument("--average", type = int, default = 1)
argparser.add_argument("--batch_size", type = int, default = 40)
argparser.add_argument("--embeddings", type = str, default = "")
argparser.add_argument("--corpus", type = str)
argparser.add_argument("--train", type = str, default = "")
argparser.add_argument("--test", type = str, default = "")
argparser.add_argument("--dev", type = str, default = "")
argparser.add_argument("--hidden_dim", "-d", type = int, default = 200)
argparser.add_argument("--embed_dim", "-e", type = int, default = 200)
argparser.add_argument("--optimizer", type = str, default = "adam")
argparser.add_argument("--learning_rate", "-lr", type = float, default = 0.001)
argparser.add_argument("--l2_reg", type = float, default = 1e-5)
argparser.add_argument("--activation", "-act", type = str, default = "tanh")
argparser.add_argument("--depth", type = int, default = 1)
argparser.add_argument("--dropout", type = float, default = 0.0)
argparser.add_argument("--max_epoch", type = int, default = 20)
argparser.add_argument("--max_seq_len", type = int, default = 100)
argparser.add_argument("--normalize", type = int, default = 1)
argparser.add_argument("--margin", type = float, default = 0.3)
argparser.add_argument("--reweight", type = int, default = 0)
argparser.add_argument("--verbose", type = int, default = 0)
argparser.add_argument("--cut_off", type = int, default = 2) # original code default 1
argparser.add_argument("--layer", type = str, default = "lstm")
argparser.add_argument("--kernel_size", type = int, default = 3)
argparser.add_argument("--save_model", type = int, default = 0)
argparser.add_argument("--cuda", type = int, default = 0)
args = argparser.parse_args('--layer lstm --corpus ../../data/askubuntu/text_tokenized.txt.gz --embeddings ../../data/askubuntu/vector/vectors_pruned.200.txt.gz --train ../../data/askubuntu/train_random.txt --dev ../../data/askubuntu/dev.txt --test ../../data/askubuntu/test.txt --l2_reg 0 -d 100 --batch_size 16 -lr 0.001'.split())
print args
print ""
#main(args)

Namespace(activation='tanh', average=1, batch_size=16, corpus='../../data/askubuntu/text_tokenized.txt.gz', cuda=0, cut_off=2, depth=1, dev='../../data/askubuntu/dev.txt', dropout=0.0, embed_dim=200, embeddings='../../data/askubuntu/vector/vectors_pruned.200.txt.gz', hidden_dim=100, kernel_size=3, l2_reg=0.0, layer='lstm', learning_rate=0.001, margin=0.3, max_epoch=20, max_seq_len=100, normalize=1, optimizer='adam', reweight=0, save_model=0, test='../../data/askubuntu/test.txt', train='../../data/askubuntu/train_random.txt', verbose=0)



In [35]:
raw_corpus = read_corpus(args.corpus)
#print 'type of raw_corpus', type(raw_corpus), '\n', raw_corpus.keys()[1]#, '\n', raw_corpus.values()[1]

# # create embedding layer
#embeddings = 'askubuntu/vector/vectors_pruned.200.txt.gz'
embedding_layer = create_embedding_layer(
            raw_corpus,
            n_d = args.embed_dim,#200,
            cut_off = 2,
            embs = load_embedding_iterator(args.embeddings) if args.embeddings else None
        )

0 empty titles ignored.
100406 pre-trained embeddings loaded.


In [36]:
ids_corpus = map_corpus(raw_corpus, embedding_layer, max_len = args.max_seq_len)
#ids_corpus['378466']

say("vocab size={}, corpus size={}\n".format(embedding_layer.n_V,len(raw_corpus)))
# because there are some out-of-vocabulary words.

padding_id = embedding_layer.vocab_map["<padding>"]
#print 'padding_id', padding_id

vocab size=100408, corpus size=167765


In [37]:
if args.reweight:
    print 'add weights to the model'
    weights = create_idf_weights(args.corpus, embedding_layer)

if args.dev:
    dev = read_annotations(args.dev, K_neg=-1, prune_pos_cnt=-1)
    dev_batches = create_eval_batches(ids_corpus, dev, padding_id, pad_left = not args.average)
if args.test:
    test = read_annotations(args.test, K_neg=-1, prune_pos_cnt=-1)
    test_batches = create_eval_batches(ids_corpus, test, padding_id, pad_left = not args.average)

In [44]:
# # Create training batches
train = read_annotations(args.train)
#batch_size = 16
train_batches = create_batches(ids_corpus, train[:50], args.batch_size, padding_id, pad_left = not args.average)
#print 'train_batch[0][0]', len(train_batches[0][0]), len(train_batches[0][1]), len(train_batches[0][2])
say("create batches\n")
say("{} batches, {} tokens in total, {} triples in total\n".format(
        len(train_batches),
        sum(len(x[0].ravel())+len(x[1].ravel()) for x in train_batches),
        sum(len(x[2].ravel()) for x in train_batches)
    ))

#train_batches = None

# initialize the model
model = Model(args, weights=weights if args.reweight else None)
optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

# get the number of parameters of the model
num_params = 0
for layer in model.parameters():
    print type(layer), layer.data.shape, len(layer.data.numpy().ravel())
    num_params += len(layer.data.numpy().ravel())
say("num of parameters: {}\n".format(num_params))

if args.cuda:
    model.cuda()

train_eval = read_annotations(args.train, K_neg=-1, prune_pos_cnt=-1)
train_eval = create_eval_batches(ids_corpus, train_eval, padding_id, pad_left = not args.average)

create batches
4 batches, 140386 tokens in total, 1386 triples in total
current model is LSTM
<class 'torch.nn.parameter.Parameter'> torch.Size([400, 200]) 80000
<class 'torch.nn.parameter.Parameter'> torch.Size([400, 100]) 40000
<class 'torch.nn.parameter.Parameter'> torch.Size([400]) 400
<class 'torch.nn.parameter.Parameter'> torch.Size([400]) 400
num of parameters: 120800
create_eval_batches if length greater than a threshold 1501


In [ ]:
for i in range(args.max_epoch):
    start_time = time.time()

    #train = read_annotations('askubuntu/train_random.txt')
    train_batches = create_batches(ids_corpus, train, args.batch_size,
                                padding_id, pad_left = not args.average)

    N =len(train_batches)
    print "num of batches", N
    #avrg_loss = 0
    #avrg_cost = 0
    for j in xrange(N):
        # get current batch
        idts, idbs, idps = train_batches[j]
        optimizer.zero_grad()

        if args.layer == 'lstm':
            model.hidden_t = model.init_hidden(idts.shape[1])
            model.hidden_b = model.init_hidden(idbs.shape[1])

        # embedding layer
        xt = embedding_layer.forward(idts.ravel()) # flatten
        xt = xt.reshape((idts.shape[0], idts.shape[1], args.embed_dim))
        xt = Variable(torch.from_numpy(xt).float())

        xb = embedding_layer.forward(idbs.ravel())
        xb = xb.reshape((idbs.shape[0], idbs.shape[1], args.embed_dim))
        xb = Variable(torch.from_numpy(xb).float())

        # build mask
        mt = np.not_equal(idts, padding_id).astype('float')
        mt = Variable(torch.from_numpy(mt).float().view(idts.shape[0], idts.shape[1], 1))

        mb = np.not_equal(idbs, padding_id).astype('float')
        mb = Variable(torch.from_numpy(mb).float().view(idbs.shape[0], idbs.shape[1], 1))

        # back prop
        h_final = model(xt.cuda(), xb.cuda(), mt.cuda(), mb.cuda())       # lstm output
        idps = Variable(torch.from_numpy(idps).long())

        loss, cost, prct = customized_loss(args, h_final, idps.cuda(), model)
        cost.backward()                              # backpropagation, compute gradients
        optimizer.step()